In [5]:
import pandas as pd
import numpy as np

input = "../data/English/ASSET_train.csv"

df = pd.read_csv(input)
df.head(5)

,original,simple
0,Celestus microblepharis is a species of lizard...,Celestus microblepharis is a type of lizard in...
1,"However, admirals have always needed additiona...",Admirals always needed more space; a large me...
2,"When Bonham Carter was just five, her mother h...",Bonham Carters mother had a three year long ne...
3,"Such is in addition, in the coats of arms of s...",this is part of the coats of arms for neighbor...
4,The new government in 1816 created the distric...,The new government in 1816 created the areas S...


In [6]:
with open("./laser_embeddings/ASSET_train.txt", 'w') as f:
    for txt in df['original']:
        f.write(txt + '\n')


In [3]:
%env LASER=../../../LASER

env: LASER=../../../LASER


In [7]:
# !bash ../../../LASER/tasks/embed/embed.sh ./laser_embeddings/simplifyUR_test.txt ./laser_embeddings/simplifyUR_test.bin urd_Arab
!bash ../../../LASER/tasks/embed/embed.sh ./laser_embeddings/ASSET_train.txt ./laser_embeddings/ASSET_train.bin

2022-12-23 17:34:51,484 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-12-23 17:34:51,539 | INFO | embed | spm_model: /Users/michaelryan/Documents/School/GeorgiaTech/Research/LASER/nllb/laser2.spm
2022-12-23 17:34:51,539 | INFO | embed | spm_cvocab: /Users/michaelryan/Documents/School/GeorgiaTech/Research/LASER/nllb/laser2.cvocab
2022-12-23 17:34:51,539 | INFO | embed | loading encoder: /Users/michaelryan/Documents/School/GeorgiaTech/Research/LASER/nllb/laser2.pt
2022-12-23 17:34:51,817 | INFO | preprocess | SPM processing ASSET_train.txt  
2022-12-23 17:34:52,210 | INFO | embed | encoding /var/folders/zy/zlsw34jx4zn2cv4tn33_02nh0000gn/T/tmp3ww2w8_c/spm to ./laser_embeddings/ASSET_train.bin
2022-12-23 17:35:44,412 | INFO | embed | encoded 10000 sentences
2022-12-23 17:36:35,304 | INFO | embed | encoded 20000 sentences
2022-12-23 17:36:35,304 | INFO | embed | encoded 20000 sentences in 103s


In [149]:
dim = 1024

train_embeddings = np.fromfile("./laser_embeddings/SimplifyUR_train.bin", dtype=np.float32, count=-1)                                                                          
train_embeddings.resize(train_embeddings.shape[0] // dim, dim)

test_embeddings = np.fromfile("./laser_embeddings/SimplifyUR_test.bin", dtype=np.float32, count=-1)                                                                          
test_embeddings.resize(test_embeddings.shape[0] // dim, dim)

In [150]:
from sklearn.neighbors import NearestNeighbors

K=500

model = NearestNeighbors(n_neighbors=K,
                         metric='cosine',
                         algorithm='brute',
                         n_jobs=-1)
model.fit(train_embeddings)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=500)

In [151]:
closest_neighbors = model.kneighbors(test_embeddings, return_distance=False)

In [152]:
closest_neighbors

array([[335, 156, 189, ..., 245, 375, 376],
       [382, 381,  90, ..., 120,  46, 489],
       [269, 219,  25, ..., 263,  93, 345],
       ...,
       [189, 208, 158, ..., 413, 494, 333],
       [433,  75, 392, ..., 497, 496, 345],
       [ 98, 563, 562, ..., 319, 320, 418]])

In [153]:
A = train_embeddings[closest_neighbors, :]
B = test_embeddings

dot_product = np.dot(A, B.T).diagonal(0,0,2).T

# Compute the L2 norm of the vectors in A and B
norm_A = np.linalg.norm(A, axis=2)
norm_B = np.linalg.norm(B, axis=1)

# Compute the cosine distance between each pair of vectors using broadcasting
cosine_distances = 1 - (dot_product / (norm_A.T * norm_B).T)

In [156]:
pd.DataFrame(closest_neighbors).to_csv("./few_shot_preprocessing/SimplifyUR_sim_experiment.csv")

In [157]:
pd.DataFrame(cosine_distances).to_csv("./few_shot_preprocessing/SimplifyUR_sim_experiment_dist.csv")

In [148]:
from numpy.random import default_rng

rng = np.random.default_rng(3600)

print(closest_neighbors.shape)
random_neighbors = rng.integers(low=0, high=train_embeddings.shape[0], size=(test_embeddings.shape[0], K))
print(random_neighbors)

(359, 20)
[[16552  4208 14108 ... 17468  7609  3436]
 [ 3275  7358 12997 ...  9352 13374 19296]
 [ 7848 16277  6437 ...  1672 10881  4506]
 ...
 [ 4317  1813  7805 ... 13294  8290 18498]
 [ 4960  1819 13176 ...  6402  2728 14712]
 [ 5282  3163  9330 ... 11535 16648 17087]]


In [165]:
def find_average_distance(distances, k=3, offset=0):
    return np.average(np.average(distances[:,offset:offset+k], axis=1))

In [168]:
find_average_distance(cosine_distances, k=5, offset=0)

0.26151112
